In [ ]:
from dotenv import load_dotenv

load_dotenv()

True

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from IPython.display import display, Markdown

# 프롬프트 템플릿 정의
prompt = ChatPromptTemplate.from_messages([
	("system", "당신은 파이썬 코드 작성을 도와주는 AI 어시스턴트 입니다."),
	("human", "{user_input}")
])

# LLM 모델 정의
model = ChatOpenAI(
	  model="gpt-4.1-nano", 
    temperature=0.3, 
)

# 프롬프트 템플릿 + llm model + 출력파서를 연결하여 체인 생성
chain = prompt | model | StrOutputParser()

# 체인 실행
response = chain.invoke({
	"user_input": "파이썬에서 리스트를 정렬하는 방법은 뭐야?"
})

# AI의 응답을 출력
print(response)

# 마크다운 출력
display(Markdown(response))

## Gradio ChatInterface
- 설치 : `uv add gradio`

In [ ]:
## 기본구조

import gradio as gr

def chat_function(message, history):
	# 메시지 출력
	print(f'입력 메시지 : {message}')
	# 채팅 히스토리 출력
	print('채팅 스토리 메시지 : ')
	for chat in history:
		print(f'사용자 : {chat['role']}, 메시지: {chat['content']}')
	return "응답메시지"

# 챗봇 인터페이스 생성
demo = gr.ChatInterface(
	fn=chat_function, # 실행할 함수
	analytics_enabled=False, # 사용자 정보를 gradio에 제공 여부
)

# 챗봇 인터페이스 실행
demo.launch()


In [ ]:
## 간단한 예제: Echo 챗봇
import gradio as gr

def echo_bot(message, history): 
	return f"당신이 입력한 메시지: {message}"

# 챗봇 인터페이스 생성
demo = gr.ChatInterface(
	fn=echo_bot,
	title="Echo Chatbot",
	description="입력한 메시지를 그대로 되돌려 주는 챗봇입니다.",
	analytics_enabled=False
)

demo.launch()

In [ ]:
## 추가 입력 컴포넌트 - 최대 응답 길이 등 기타 설정을 위한 추가 입력

## 실습 프로젝트
### **다음과 같은 요구사항을 Gradio ChatInterface로 구현합니다**

- 주제: 맞춤형 여행 일정 계획 어시스턴트
- 기능: 
   - OpenAI Chat Completion API와 LangChain을 활용하여 사용자의 선호도에 맞는 여행 일정을 생성
   - LCEL을 사용하여 단계별 프롬프트 체인 구성 (사용자 입력 분석 -> 일정 생성 -> 세부 계획 수립)
   - 채팅 히스토리 사용하여 답변 생성
   - Gradio 인터페이스를 통해 사용자와 대화형으로 상호작용

- 주요 포인트:

   1. **모델 매개변수 최적화**
      - temperature=0.7: 적당한 창의성을 유지하면서 일관된 응답 생성
      - top_p=0.9: 높은 확률의 토큰만 선택하여 응답의 품질 향상
      - presence_penalty와 frequency_penalty: 반복적인 응답을 줄이고 다양한 제안 생성

   2. **시스템 프롬프트 설계**
      - 여행 플래너로서의 역할과 응답 가이드라인을 명확히 정의
      - 구체적인 정보를 포함하도록 지시
      - 한국어 응답 명시

   3. **메모리 관리**
      - Gradio 또는 LangChain 메모리 기능을 사용하여 대화 컨텍스트 유지
      - 이전 대화 내용을 바탕으로 연속성 있는 응답 생성

### 단계별 구현 가이드

- **Step 1**: 시스템 프롬프트 작성
   - 여행 플래너의 역할 명시
   - 구체적인 정보 포함 지시 (날짜, 장소, 예산 등)

- **Step 2**: 채팅 히스토리 관리
   - MessagesPlaceholder 사용
   - HumanMessage/AIMessage 변환

- **Step 3**: Gradio 인터페이스 설정
   - 제목과 설명 추가

**Step 4**: 테스트
   - "서울에서 2박 3일 여행 계획 짜줘" 등의 질문 시도

In [ ]:
import gradio as gr
from langchain_openai import ChatOpenAI
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import AIMessage, HumanMessage
from langchain_core.output_parsers import StrOutputParser
from pydantic import BaseModel, Field
from typing import Optional

# 메시지 플레이스 홀더가 있는 프롬프트 템플릿 정의
# 1. 입력 정리 프롬프트 - 날짜, 장소, 예산
class TravelPreference(BaseModel):
	"""여행 기본 정보"""
	place: str = Field(description="여행 장소(도시/지역)")
	date: str = Field(description="여행 날짜/기간 (예: 2026-02-01~2026-02-07 또는 2박 3일)")
	budget: Optional[int] = Field(default=None, description="예산(원 단위 정수), 없으면 None")


input_check_prompt = ChatPromptTemplate.from_messages([
	("system", 
	"너는 맞춤형 여행 일정 계획 어시스턴트야.\n"
	"사용자 입력과 chat_history에 입력된 대화를 보고 여행 기본 정보를 출력해.\n"
	"출력은 반드시 TravelPreference에 맞는 값으로 출력해\n"
	"예산은 원 단위 정수로, 모르면 null로 둬\n"
	"모든 내용은 한국어 기준으로 작성해!\n"
	),
	MessagesPlaceholder("chat_history"),
	("human", "{user_input}")
])

# 2. 여행 기본정보 프롬프트
schedule_draft_prompt = ChatPromptTemplate.from_messages([
    ("system",
		"너는 맞춤형 여행 일정 계획 어시스턴트야.\n"
    "입력된 여행 기본 정보(place/date/budget)를 바탕으로 '일정 초안'을 만들어.\n"
    "반드시 한국어로.\n"
    "형식: Day 1 / Day 2 ... 각 Day는 아침/점심/저녁/야간(선택).\n"
    "예산이 있으면 대략 배분도 포함해."
    ),
    MessagesPlaceholder("chat_history"),
    ("human", "여행 기본 정보:\n{pref}\n\n사용자 요청:\n{user_input}")
])

# 3. 여행 현실적 스케줄 프롬프트
schedule_detail_prompt = ChatPromptTemplate.from_messages([
    ("system",
    "너는 맞춤형 여행 일정 계획 어시스턴트야.\n"
    "'일정 초안'을 더 실전적으로 다듬어.\n"
    "각 Day마다: 이동 팁 1개, 예약/대기 팁 1개, 대안 코스 1개를 추가해.\n"
    "마지막에 확인 질문 2개만 해."
    ),
    MessagesPlaceholder("chat_history"),
    ("human", "일정 초안:\n{draft}\n\n사용자 요청:\n{user_input}")
])

# LLM 모델 정의
def chat_model_function(model, temperature):
		if model == 'gpt-4.1-nano':
			return ChatOpenAI(
				model=model, 
				temperature=temperature,
				top_p=0.9,
				presence_penalty=0.3,
				frequency_penalty=0.3,
			)
		elif model == 'gemini-2.5-flash-lite':
			return ChatGoogleGenerativeAI(
				model=model, 
				temperature=temperature,
			)
		else:
			raise ValueError("지원하지 않는 모델: {model}")
	

# 프롬프트 템플릿 + LLM 모델 + 출력파서를 연결하여 체인 생성
parser = StrOutputParser()

llm = chat_model_function("gpt-4.1-nano", 0.7)

input_check_chain = input_check_prompt | llm.with_structured_output(TravelPreference)
schedule_draft_chain = schedule_draft_prompt | llm | parser
schedule_detail_chain = schedule_detail_prompt | llm | parser


# 사용자 메시지를 처리하고 AI 응답을 생성하는 함수 (chat_history 사용)
def answer_invoke(message, history):
    history_messages = []
    for msg in history:
        if msg['role'] == "user":
            history_messages.append(HumanMessage(content=msg['content']))
        elif msg['role'] == "assistant":
            history_messages.append(AIMessage(content=msg['content']))

    history_messages.append(HumanMessage(content=message))
    
		# Step1: 스키마 객체로 입력 정리
    pref = input_check_chain.invoke({
        "chat_history": history_messages,
        "user_input": message
    })  # pref: TravelPreference

    # Step2: 일정 초안
    draft = schedule_draft_chain.invoke({
        "chat_history": history_messages,
        "pref": pref.model_dump(),   # dict로 넘기면 충분히 잘 읽음
        "user_input": message
    })

    # Step3: 일정 디테일
    final = schedule_detail_chain.invoke({
        "chat_history": history_messages,
        "draft": draft,
        "user_input": message
    })

    return final
# 사용자 메시지를 처리하고 AI 응답을 생성하는 함수 (chat_history 사용)

# Gradio ChatInterface 객체 생성
demo = gr.ChatInterface(
	fn=answer_invoke,         # 메시지 처리 함수
	title="맞춤형 여행 일정 계획 어시스턴트", # 채팅 인터페이스의 제목
  analytics_enabled=False
)

# Gradio 인터페이스 실행
demo.launch()


* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.


In [11]:
demo.close()

Closing server running on port: 7860
